In [5]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [10]:
from scripts.data_visualization import *
from scripts.data_cleaning import *
from scripts.euclidean_distance import *
from scripts.load_data import *
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

data = load_data_using_sqlalchemy(query)

if data is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

An error occurred: DLL load failed while importing _psycopg: The specified module could not be found.
Failed to load data.


In [ ]:
engagement_columns = ['sessions_frequency', 'session_duration', 'session_total_traffic']
engagement_data = data[engagement_columns].copy()
scaler = StandardScaler()
engagement_data_scaled = scaler.fit_transform(engagement_data)

In [ ]:
kmeans_engagement = KMeans(n_clusters=3, random_state=42)
data['engagement_cluster'] = kmeans_engagement.fit_predict(engagement_data_scaled)
least_engaged_cluster_center = kmeans_engagement.cluster_centers_[data['engagement_cluster'].value_counts().idxmin()]

In [ ]:
data['engagement_score'] = data[engagement_columns].apply(
    lambda row: calculate_euclidean_distance(row, least_engaged_cluster_center), axis=1
)

In [ ]:
experience_columns = ['avg_tcp_retrans', 'avg_rtt', 'avg_throughput']
experience_data = data[experience_columns].copy()
experience_data_scaled = scaler.fit_transform(experience_data)

In [ ]:
kmeans_experience = KMeans(n_clusters=3, random_state=42)
data['experience_cluster'] = kmeans_experience.fit_predict(experience_data_scaled)
worst_experience_cluster_center = kmeans_experience.cluster_centers_[data['experience_cluster'].value_counts().idxmax()]

In [ ]:
data['experience_score'] = data[experience_columns].apply(
    lambda row: calculate_euclidean_distance(row, worst_experience_cluster_center), axis=1
)

In [ ]:
data['satisfaction_score'] = data[['engagement_score', 'experience_score']].mean(axis=1)
top_10_satisfied = data.nlargest(10, 'satisfaction_score')[['MSISDN/Number', 'satisfaction_score']]

In [ ]:
X = data[engagement_columns + experience_columns]
y = data['satisfaction_score']
model = LinearRegression()
model.fit(X, y)

In [ ]:
data['predicted_satisfaction'] = model.predict(X)

In [ ]:
score_data = data[['engagement_score', 'experience_score']]
kmeans_scores = KMeans(n_clusters=2, random_state=42)
data['satisfaction_cluster'] = kmeans_scores.fit_predict(score_data)

In [ ]:
cluster_summary = data.groupby('satisfaction_cluster').agg(
    avg_satisfaction=pd.NamedAgg(column='satisfaction_score', aggfunc='mean'),
    avg_experience=pd.NamedAgg(column='experience_score', aggfunc='mean')
)

In [ ]:
cursor = load_data_from_postgres(query)

In [ ]:
cursor.execute
CREATE TABLE IF NOT EXISTS user_satisfaction (
    user_id VARCHAR(50),
    engagement_score FLOAT,
    experience_score FLOAT,
    satisfaction_score FLOAT
)

In [ ]:
for _, row in data[['MSISDN/Number', 'engagement_score', 'experience_score', 'satisfaction_score']].iterrows():
    cursor.execute("""
    INSERT INTO user_satisfaction (user_id, engagement_score, experience_score, satisfaction_score)
    VALUES (%s, %s, %s, %s)
    """, tuple(row))

connection.commit()

In [ ]:
cursor.execute("SELECT * FROM user_satisfaction LIMIT 10")
output = cursor.fetchall()
print("Sample Data from MySQL Table:")
for record in output:
    print(record)

In [ ]:
connection.close()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(data['engagement_score'], data['experience_score'], c=data['satisfaction_cluster'], cmap='viridis')
plt.title("Clustering of Engagement & Experience Scores")
plt.xlabel("Engagement Score")
plt.ylabel("Experience Score")
plt.colorbar(label='Cluster')
plt.show()